<a href="https://colab.research.google.com/github/heartzhacker/Emotion-classification/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing the necessary libraries**



In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import zipfile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
local_data_folder = "/content/Data"
os.makedirs(local_data_folder, exist_ok=True)

###Importing the **RAVDESS** dataset saved in my Gdrive as a zip file

In [4]:
data_zip_file = "/content/drive/MyDrive/data.zip"

with zipfile.ZipFile(data_zip_file, 'r') as zip_ref:
    zip_ref.extractall(local_data_folder)

Dividing each video in 20 frames per second

In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder, target_fps=20):
    cap = cv2.VideoCapture(video_path)
    video_fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    every_nth_frame = video_fps // target_fps

    os.makedirs(output_folder, exist_ok=True)

    frame_number = 0
    extracted_frames = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_number % every_nth_frame == 0:
            frame_file = os.path.join(output_folder, f"frame{extracted_frames:04d}.png")
            cv2.imwrite(frame_file, frame)
            extracted_frames += 1

        frame_number += 1

    cap.release()

data_folder = "/content/Data/data"
output_base_folder = "/content/Output_RAVDESS"

for actor in range(1, 25):
    actor_folder = f"Actor_{actor:02d}"
    actor_data_folder = os.path.join(data_folder, actor_folder)
    actor_output_folder = os.path.join(output_base_folder, actor_folder)

    file_list = os.listdir(actor_data_folder)

    for file in file_list:
        if file.endswith(".mp4"):
            video_path = os.path.join(actor_data_folder, file)
            video_output_folder = os.path.join(actor_output_folder, os.path.splitext(file)[0])
            extract_frames(video_path, video_output_folder, target_fps=20)